# DiploDatos 2021


### Categorización de publicaciones de productos realizadas en Mercado Libre

### 02 - Análisis y Curación

#### Condiciones generales que aplican a todos los prácticos:
   - Las notebooks tienen que ser 100% reproducibles, es decir al ejecutar las celdas tal cuál como se entrega la notebook se deben obtener los mismos resultados sin errores.
   - Código legible, haciendo buen uso de las celdas de la notebook y en lo posible seguir estándares de código para Python (https://www.python.org/dev/peps/pep-0008/).
   - Utilizar celdas tipo "Markdown" para ir guiando el análisis.
   - Limpiar el output de las celdas antes de entregar el notebook (ir a Kernel --> Restart Kernel and Clear All Ouputs).
   - Incluir conclusiones del análisis que se hizo en la sección "Conclusiones". Tratar de aportar valor en esta sección, ser creativo! 

## 1. Consignas

#### Sección C: Label Encoding

1. Utilizar método *LabelEncoder()* de sklearn:

- https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html

#### Sección D: Word Embeddings

Generar los word embeddings correspondientes, de las siguientes 2 formas: 

1. Custom Word Embeddings
2. Loading Pretrained Word Embeddings (opcional)

En ambos puntos el objetivos final es llegar a crear la embedding layer de keras:

- https://keras.io/api/layers/core_layers/embedding/

## 3. Conclusiones

#### Material de ayuda para el desarrollo de este práctico:

1. Implementación en keras de word embeddings: https://stackabuse.com/python-for-nlp-word-embeddings-for-deep-learning-in-keras
2. Como utilizar pre-trained word embeddings en Keras: https://keras.io/examples/nlp/pretrained_word_embeddings/
3. Word Embeddings: https://jalammar.github.io/illustrated-word2vec/
3. Curso de procesamiento del lenguaje natural con Keras: https://www.coursera.org/learn/natural-language-processing-tensorflow/home/welcome

## 2. Código y análisis

Imports necesarios

In [3]:
!pip3 install unidecode 


     |████████████████████████████████| 245kB 6.4MB/s 


In [18]:


import pandas as pd
import numpy
from unidecode import unidecode
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download("stopwords")
nltk.download('punkt')

stopwords = \
    set(nltk.corpus.stopwords.words("spanish")) | \
    set(nltk.corpus.stopwords.words("portuguese"))

# Función que hace el Unidecode
def uni_deco(s: str):
    return unidecode(s)

def remove_unimportant_words(s):
    """
    Removes from the string @s all the stopwords, digits, and special chars
    """
    special_chars = "-.+,[@_!#$%^&*()<>?/\|}{~:]"
    digits = "0123456789"
    invalid_chars = special_chars + digits

    reduced_title = ''.join(c for c in s if not c in invalid_chars)
    
    reduced_title = ' '.join(
        w.lower() for w in word_tokenize(reduced_title)
        if not w.lower() in stopwords
    )
    return reduced_title


# Si después vamos a quitar los stopwords, quizás habría que directamente 
# quitar las contracciones en lugar de reemplazarlas.

def contracciones(s: str):
    title=s
    dict_contracciones={" c/u ": "cada uno", " p/":"para"," c/":"con"}
    for key,value in dict_contracciones.items():
        title=title.split(key)
        title=value.join(title)
    return title

#print(unidecode(u"\u5317\u4EB0"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
# Lectura del Dataset reducido
URL = "https://www.famaf.unc.edu.ar/~nocampo043/ml_challenge2019_dataset.csv"
df = pd.read_csv(URL)

In [6]:
df.head()

,title,label_quality,language,category
0,Galoneira Semi Industrial,unreliable,portuguese,SEWING_MACHINES
1,Máquina De Coser Brother Industrial,unreliable,spanish,SEWING_MACHINES
2,Teclado Casio Wk-240 76 Teclas Profissional St...,unreliable,portuguese,MUSICAL_KEYBOARDS
3,Heladera Gafa 380 Impecable Urgente,unreliable,spanish,REFRIGERATORS
4,Butaca 6 Cuotas Sin Interes!! Para Auto Bebes...,unreliable,spanish,BABY_CAR_SEATS


Lectura de dataset reducido

#### Sección A:  Limpieza de texto / Preprocessing

Tener en cuenta lo siguiente: 

1. Unidecode

2. Pasar a minúsculas

3. Limpiar números

4. Limpiar símbolos --> '!¡"@$%&*,.:;<=>?@[\\]^_`{|}~\t\n'

5. Limpiar caracteres que sueles usarse como espacios --> '+()-\''

6. Reemplazar "contracciones", por ejemplo:
        -'c/u ' por 'cada uno'
        -'c/' por 'con'
        -'p/' por 'para'
        
        
7. Etc! Pueden sumar mas steps

***A.1 Unidecode***

In [7]:
df.title.apply(uni_deco).apply(contracciones).apply(remove_unimportant_words)

0                                 galoneira semi industrial
1                          maquina coser brother industrial
2             teclado casio wk teclas profissional standard
3                           heladera gafa impecable urgente
4                     butaca cuotas interes auto bebesit kg
                                ...                        
646755                                   thank you malbec x
646756    cachorros jack rusell terrier corto ultimo dis...
646757    colchao box casal castor vitagel euro one face xx
646758                                maquina cortar starex
646759           trimmer detailer wahl kit tijeras stylecut
Name: title, Length: 646760, dtype: object

In [8]:
df[df.title.apply(lambda s: "/" in s)][df.language=="portuguese"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,title,label_quality,language,category
74,Cartão Evo Select Samsung Microsd 128gb U3 4k ...,unreliable,portuguese,MEMORY_CARDS
82,Cartão Sdhc Sandisk Extreme Classe 10 32gb 90...,reliable,portuguese,MEMORY_CARDS
93,Patins Rollerblade K2 Semi Novo 42/43,unreliable,portuguese,ROLLER_SKATES
124,Furadeira De Impacto Mondial 3/8 550 Watts Fi-09,reliable,portuguese,ELECTRIC_DRILLS
142,Máquina De Cortar Cabelo/barba Cabeleireiro Ph...,reliable,portuguese,HAIR_CLIPPERS
...,...,...,...,...
646663,Patins 3 Rodas Azul Ajust. 26/29 Kit Proteção ...,unreliable,portuguese,ROLLER_SKATES
646687,Cartão Memória Micro Sd 4gb Lacrado Original C...,unreliable,portuguese,MEMORY_CARDS
646693,Respiração 3d Almofada Assento De Malha / Almo...,unreliable,portuguese,BABY_CAR_SEATS
646704,Furadeira De Impacto Sdh600-br 1/2 600w - Sta...,unreliable,portuguese,ELECTRIC_DRILLS


In [ ]:
df.loc[142].title

'Máquina De Cortar Cabelo/barba Cabeleireiro Philips Hc3410'

#### Sección B: Tokenización & Secuencias

1. Utilizar métodos *fit_on_texts()*, *texts_to_sequences()* y *pad_sequences()*:

- https://keras.io/api/preprocessing/text/#tokenizer-class

- https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences

In [37]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


In [38]:

tokenizer = Tokenizer(lower=True)#pruebas
tok=tokenizer.fit_on_texts(df.title) #pruebas

In [34]:
X = tokenizer.texts_to_sequences(df.title) #pruebas



In [47]:

dictionary=dict()
rev_dictionary = dict()
for key, value in tokenizer.word_index.items():
    dictionary[value] = key
    rev_dictionary[key] = value


In [43]:
tokenizer.word_index.items()

dict_items([('de', 1), ('para', 2), ('2', 3), ('1', 4), ('com', 5), ('cocina', 6), ('4', 7), ('con', 8), ('kit', 9), ('3', 10), ('teclado', 11), ('a', 12), ('maquina', 13), ('e', 14), ('máquina', 15), ('micro', 16), ('y', 17), ('c', 18), ('sd', 19), ('heladera', 20), ('inox', 21), ('jaqueta', 22), ('colchão', 23), ('patins', 24), ('carrinho', 25), ('calça', 26), ('bebê', 27), ('5', 28), ('cafetera', 29), ('10', 30), ('jeans', 31), ('taladro', 32), ('x', 33), ('furadeira', 34), ('auto', 35), ('short', 36), ('cafeteira', 37), ('parede', 38), ('mala', 39), ('relógio', 40), ('campera', 41), ('bebe', 42), ('original', 43), ('fogão', 44), ('bacha', 45), ('220v', 46), ('vinho', 47), ('cartão', 48), ('industrial', 49), ('reloj', 50), ('s', 51), ('sandisk', 52), ('6', 53), ('preto', 54), ('bermuda', 55), ('memoria', 56), ('valija', 57), ('bocas', 58), ('talle', 59), ('johnson', 60), ('impacto', 61), ('butaca', 62), ('cuba', 63), ('viagem', 64), ('cm', 65), ('acero', 66), ('em', 67), ('0', 68), 

Con la función prepare_tokenizer , recibe el dataframe y retorna una lista de pabras con la más frecuente.

In [64]:
def prepare_tokenizer(words):
    '''
        funcion que genera un vocabulario, toma una lista de palabras.
        retorna una lista de palabras tokenizadas.
    '''
    # obtain a tokenizer
    t = Tokenizer(filters = '-.+,[@_!#$%^&*()<>?/\|}{~:]0123456789', lower=True) # filters and lower
    t.fit_on_texts(words) #Actualiza el vocabulario interno basado en una lista de textos.
    dictionary = dict(); rev_dictionary = dict()#definimos diccionarios
    #recorremos todos los elementos, palabra por palabra.
    for key, value in t.word_index.items():
        dictionary[value] = key
        rev_dictionary[key] = value
    #tamaño del diccionario de frecuencias.
    vocab_size = len(t.word_index) + 1

    ''' Small modification from Animesh
        # also add the '<unk>' token to the dictionary at 0th position
    '''
    dictionary[0] = '<unk>'; rev_dictionary['<unk>'] = 0

    #print (vocab_size) # size vocab
	# integer encode the documents
    encoded_docs = t.texts_to_sequences(words)

    return (dictionary, rev_dictionary, vocab_size)

In [65]:
prepare_tokenizer(df.title)

({1: 'de',
  2: 'x',
  3: 'para',
  4: 'v',
  5: 'a',
  6: 'gb',
  7: 'com',
  8: 'cocina',
  9: 'con',
  10: 'cm',
  11: 'e',
  12: 'kit',
  13: 'teclado',
  14: 'c',
  15: 'maquina',
  16: 'w',
  17: 'y',
  18: 'máquina',
  19: 'micro',
  20: 's',
  21: 'sd',
  22: 'inox',
  23: 'heladera',
  24: 'jaqueta',
  25: 'colchão',
  26: 'patins',
  27: 'carrinho',
  28: 'calça',
  29: 'bebê',
  30: 'cafetera',
  31: 'kg',
  32: 'jeans',
  33: 'taladro',
  34: 'furadeira',
  35: 'auto',
  36: 'short',
  37: 'cafeteira',
  38: 'parede',
  39: 'mala',
  40: 'relógio',
  41: 'campera',
  42: 'bebe',
  43: 'original',
  44: 'fogão',
  45: 'bacha',
  46: 'vinho',
  47: 'cartão',
  48: 'industrial',
  49: 'd',
  50: 'reloj',
  51: 'sandisk',
  52: 'ml',
  53: 'preto',
  54: 'm',
  55: 'mm',
  56: 'bermuda',
  57: 'memoria',
  58: 'valija',
  59: 'bocas',
  60: 'talle',
  61: 'l',
  62: 'johnson',
  63: 'impacto',
  64: 'butaca',
  65: 'cuba',
  66: 'p',
  67: 'viagem',
  68: 'acero',
  69: 'em',
 